In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import common as common

In [2]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = self.dropout(x[:, -1, :])
        out = self.fc(x)
        return out

In [ ]:
train_df = common.get_train_df()
target_column, feature_column = common.get_column_names(train_df)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_df[target_column])

scaler = StandardScaler()
X = scaler.fit_transform(train_df[feature_column])

X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
y = torch.tensor(y, dtype=torch.long)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


input_size = X.shape[2]
hidden_size = 256
epochs = 100
num_classes = len(label_encoder.classes_)
model = LSTMModel(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)


for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Validation Accuracy: {accuracy:.2f}%")

Epoch [1/100], Validation Accuracy: 28.07%
Epoch [2/100], Validation Accuracy: 29.24%
Epoch [3/100], Validation Accuracy: 30.41%
Epoch [4/100], Validation Accuracy: 28.65%
Epoch [5/100], Validation Accuracy: 23.98%
Epoch [6/100], Validation Accuracy: 22.22%
Epoch [7/100], Validation Accuracy: 22.81%
Epoch [8/100], Validation Accuracy: 21.64%
Epoch [9/100], Validation Accuracy: 22.81%
Epoch [10/100], Validation Accuracy: 21.64%
Epoch [11/100], Validation Accuracy: 22.22%
Epoch [12/100], Validation Accuracy: 21.64%
Epoch [13/100], Validation Accuracy: 21.05%
Epoch [14/100], Validation Accuracy: 23.39%
Epoch [15/100], Validation Accuracy: 21.05%
Epoch [16/100], Validation Accuracy: 20.47%
Epoch [17/100], Validation Accuracy: 21.05%
Epoch [18/100], Validation Accuracy: 23.39%
Epoch [19/100], Validation Accuracy: 22.22%
Epoch [20/100], Validation Accuracy: 23.98%
Epoch [21/100], Validation Accuracy: 22.81%
Epoch [22/100], Validation Accuracy: 23.39%
Epoch [23/100], Validation Accuracy: 23.3